# Scalable Video Classification with LoRA Fine-Tuning and LangGraph-Based Video Summarization
Author: GangadharSShiva

## SECTION 1: INSTALLATION AND SETUP

In [ ]:
# Install required packages
!pip install -q langgraph typing_extensions torch torchvision peft
!pip install -q moviepy SpeechRecognition pydub
!apt-get install -y ffmpeg

## SECTION 2: IMPORTS

In [ ]:
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from peft import LoraConfig, get_peft_model
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip, concatenate_videoclips
import speech_recognition as sr
from pydub import AudioSegment
from skimage.metrics import structural_similarity as ssim
from IPython.display import Video, display
from google.colab import drive

# LangGraph Imports
from typing import TypedDict
from langgraph.graph import StateGraph, END

## SECTION 3: GOOGLE DRIVE SETUP AND DATA EXTRACTION

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths
zip_file_path_in_drive = '/content/drive/My Drive/aai-521/videos-ds.zip'
destination_folder = '/content/drive/My Drive/aai-521/videos-ds'
train_data_dir = os.path.join(destination_folder, 'train/')
test_data_dir = os.path.join(destination_folder, 'test/')
train_csv_path = os.path.join(destination_folder, 'train.csv')
test_csv_path = os.path.join(destination_folder, 'test.csv')

# Create destination folder if needed
os.makedirs(destination_folder, exist_ok=True)

# Copy and unzip data
if os.path.exists(zip_file_path_in_drive):
    !cp "{zip_file_path_in_drive}" "{destination_folder}/videos-ds.zip"
    print("✓ videos-ds.zip loaded from Google Drive")
    
    zip_file_path = os.path.join(destination_folder, "videos-ds.zip")
    !unzip -o -q "{zip_file_path}" -d "{destination_folder}"
    print("✓ Data extracted successfully")
    
    # Verify files
    avi_files = glob.glob(os.path.join(destination_folder, '**', '*.avi'), recursive=True)
    print(f"✓ Found {len(avi_files)} video files")
else:
    print("✗ Error: videos-ds.zip not found")

## SECTION 4: UTILITY FUNCTIONS (Video Frame and DataLoader)

In [ ]:
def extract_frames(video_path, num_frames=16):
    """Extract evenly-spaced frames from video."""
    frames = []
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return frames
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        cap.release()
        return frames
    
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    
    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    
    cap.release()
    return frames


def collate_fn(batch):
    """Custom collate function to filter None values and ensure consistent frame count."""
    batch = [item for item in batch 
             if item is not None and item[0] is not None and item[1] is not None]
    
    if not batch:
        return None, None
    
    inputs, labels_or_paths = zip(*batch)
    
    # Check frame count consistency
    expected_frames = 16
    if any(inp.size(0) != expected_frames for inp in inputs):
        print(f"⚠ Skipping batch: inconsistent frame count (expected {expected_frames})")
        return None, None
    
    return torch.stack(inputs), list(labels_or_paths)

## SECTION 5: DATASET CLASS

In [ ]:
class VideoDataset(Dataset):
    """Custom dataset for video classification."""
    
    def __init__(self, data_dir, csv_path, num_frames=16, transform=None, is_test=False):
        self.data_dir = data_dir
        self.transform = transform
        self.videos = []
        self.labels = []
        self.is_test = is_test
        self.video_paths = []
        
        df = pd.read_csv(csv_path)
        
        for _, row in df.iterrows():
            video_name = row['video_name']
            if not self.is_test:
                self.labels.append(row['tag'])
            
            video_path = os.path.join(data_dir, video_name)
            if os.path.exists(video_path):
                self.videos.append(video_path)
                if self.is_test:
                    self.video_paths.append(video_path)
    
    def __len__(self):
        return len(self.videos)
    
    def __getitem__(self, idx):
        video_path = self.videos[idx]
        frames = extract_frames(video_path)
        
        if not frames:
            return None, (video_path if self.is_test else None)
        
        frames = torch.from_numpy(np.array(frames)).permute(0, 3, 1, 2).float()
        
        if self.transform:
            frames = torch.stack([self.transform(frame) for frame in frames])
        
        if not self.is_test:
            return frames, self.labels[idx]
        else:
            return frames, video_path

## SECTION 6: DATA LOADING (Video Classification)

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = VideoDataset(train_data_dir, train_csv_path, transform=transform, is_test=False)
test_dataset = VideoDataset(test_data_dir, test_csv_path, transform=transform, is_test=True)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

print(f"✓ Training samples: {len(train_dataset)}")
print(f"✓ Test samples: {len(test_dataset)}")
print(f"✓ Number of classes: {len(set(train_dataset.labels))}")

## SECTION 7: MODEL SETUP WITH LORA (Video Classification)

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✓ Using device: {device}")

# Load pre-trained ResNet-18
num_classes = len(set(train_dataset.labels))
model = resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Apply LoRA
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "conv1", "layer1.0.conv1", "layer1.0.conv2", "layer1.1.conv1", "layer1.1.conv2",
        "layer2.0.conv1", "layer2.0.conv2", "layer2.0.downsample.0", "layer2.1.conv1", "layer2.1.conv2",
        "layer3.0.conv1", "layer3.0.conv2", "layer3.0.downsample.0", "layer3.1.conv1", "layer3.1.conv2",
        "layer4.0.conv1", "layer4.0.conv2", "layer4.0.downsample.0", "layer4.1.conv1", "layer4.1.conv2",
        "fc"
    ],
    lora_dropout=0.05,
    bias="none"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# Optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("✓ Model configured with LoRA")

## SECTION 8: TRAINING (OPTIONAL - UNCOMMENT TO TRAIN)

In [ ]:
# num_epochs = 10
# model.train()
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for i, data in enumerate(train_dataloader):
#         if data is None or data[0] is None:
#             continue
#         
#         inputs, labels = data
#         unique_labels = sorted(list(set(train_dataset.labels)))
#         label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
#         labels = torch.tensor([label_to_int[l] for l in labels], dtype=torch.long).to(device)
#         inputs = inputs.to(device)
#         
#         optimizer.zero_grad()
#         # Treat all frames in the batch as individual images for the 2D CNN (ResNet)
#         outputs = model(inputs.view(-1, 3, 112, 112))
#         # Average the frame-level predictions to get a video-level prediction
#         outputs = outputs.view(inputs.size(0), inputs.size(1), -1).mean(dim=1)
#         
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         
#         running_loss += loss.item()
#         if (i + 1) % 10 == 0:
#             print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {running_loss/10:.4f}')
#             running_loss = 0.0

## SECTION 9: EVALUATION (Video Classification)

In [ ]:
print("\n" + "="*80)
print("VIDEO CLASSIFICATION EVALUATION")
print("="*80)

model.eval()
predictions = []
video_paths = []

with torch.no_grad():
    for data in test_dataloader:
        if data is None or data[0] is None:
            continue
        
        inputs, paths = data
        # Use torch.stack to ensure consistent tensor structure if collate_fn returned a list of tensors
        inputs = torch.stack(inputs).to(device)
        
        outputs = model(inputs.view(-1, 3, 112, 112))
        outputs = outputs.view(inputs.size(0), inputs.size(1), -1).mean(dim=1)
        
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        video_paths.extend(paths)

# Map predictions to labels
unique_labels = sorted(list(set(train_dataset.labels)))
label_map = {i: label for i, label in enumerate(unique_labels)}
predicted_labels = [label_map[pred] for pred in predictions]

results_df = pd.DataFrame({
    'video_path': video_paths, 
    'predicted_label': predicted_labels
})

print("\n✓ Predictions on test videos:")
print(results_df.head(10))

## SECTION 10: LANGGRAPH-BASED VIDEO SUMMARIZATION
This section uses LangGraph to orchestrate the multi-step summarization process: Audio Extraction -> Transcription -> Summary Video Creation.

In [ ]:
# Define the State of the LangGraph Agent
class SummaryState(TypedDict):
    video_path: str
    audio_path: str
    summary_video_path: str
    transcript: str
    duration: float
    summary_clips_count: int
    error: str

# --- 10.1 LangGraph Nodes (Functions) ---

def extract_audio(state: SummaryState) -> SummaryState:
    """Node 1: Extracts audio from the video and updates the state."""
    print("\n▶ 1. Executing Audio Extraction...")
    video_path = state['video_path']
    audio_path = video_path.replace('.avi', '.wav')
    
    try:
        video_clip = VideoFileClip(video_path)
        video_clip.audio.write_audiofile(audio_path, logger=None)
        video_clip.close()
        print("✓ Audio extracted.")
        return {**state, "audio_path": audio_path, "error": None}
    except Exception as e:
        print(f"✗ Audio extraction failed: {e}")
        return {**state, "error": f"Audio extraction failed: {e}"}


def transcribe_audio(state: SummaryState) -> SummaryState:
    """Node 2: Transcribes the extracted audio and updates the state with the transcript."""
    print("▶ 2. Executing Transcription...")
    audio_path = state.get('audio_path')
    
    if not audio_path or state.get('error'):
        return {**state, "transcript": "Skipped due to prior error.", "error": state.get('error')}
        
    transcript = "No speech detected in video"
    try:
        r = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio_data = r.record(source)
            transcript = r.recognize_google(audio_data)
        print(f"✓ Transcript generated: {transcript[:50]}...")
        return {**state, "transcript": transcript, "error": None}
    except Exception as e:
        print(f"⚠ Transcription failed: {e} (assuming no speech)")
        return {**state, "transcript": transcript, "error": None}


def create_summary_video(state: SummaryState) -> SummaryState:
    """Node 3: Extracts key frames and creates the summary video."""
    print("▶ 3. Executing Summary Video Creation...")
    video_path = state['video_path']
    summary_video_path = video_path.replace('.avi', '_summary.mp4')
    
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()
    
    if fps == 0:
        return {**state, "error": "FPS is zero, cannot calculate duration/frames."}
        
    duration = frame_count / fps
    frame_indices = list(range(0, frame_count, 15)) # Extract frame every 15 frames
    segment_duration = 1 # 1 second clip per key frame
    summary_clips = []
    
    try:
        for frame_idx in frame_indices:
            start_time = frame_idx / fps
            end_time = min(start_time + segment_duration, duration)
            
            if end_time > start_time:
                clip = VideoFileClip(video_path).subclip(start_time, end_time)
                summary_clips.append(clip)

        if summary_clips:
            final_summary = concatenate_videoclips(summary_clips)
            final_summary.write_videofile(summary_video_path, codec='libx264', 
                                           audio_codec='aac', logger=None)
            final_summary.close()
            print(f"✓ Summary video saved: {summary_video_path}")
            
            return {**state, 
                    "summary_video_path": summary_video_path, 
                    "duration": duration, 
                    "summary_clips_count": len(summary_clips), 
                    "error": None
                   }
        else:
            return {**state, "error": "No clips extracted for summary."}
    except Exception as e:
        return {**state, "error": f"Summary creation failed: {e}"}

# --- 10.2 Build and Run the LangGraph ---

# Build the graph
workflow = StateGraph(SummaryState)

# Add nodes
workflow.add_node("extract_audio", extract_audio)
workflow.add_node("transcribe_audio", transcribe_audio)
workflow.add_node("create_summary_video", create_summary_video)

# Set edges
workflow.add_edge('extract_audio', 'transcribe_audio')
workflow.add_edge('transcribe_audio', 'create_summary_video')
workflow.add_edge('create_summary_video', END)

# Set the starting point
workflow.set_entry_point('extract_audio')

# Compile the graph
app = workflow.compile()

# Run the graph on a sample video
print("\n" + "="*80)
print("LANGGRAPH EXECUTION START")
print("="*80)

sample_video_path = '/content/drive/My Drive/aai-521/videos-ds/test/v_CricketShot_g01_c01.avi'
initial_state = {
    'video_path': sample_video_path,
    'audio_path': '',
    'summary_video_path': '',
    'transcript': '',
    'duration': 0.0,
    'summary_clips_count': 0,
    'error': None
}

final_state = app.invoke(initial_state)

# --- 10.3 Display Final Results ---

print("\n" + "="*80)
print("LANGGRAPH FINAL SUMMARY OUTPUT")
print("="*80)

if final_state.get('error'):
    print(f"❌ ERROR: {final_state['error']}")
else:
    print(f"Video Path: {final_state['video_path']}")
    print(f"Transcript: {final_state['transcript']}")
    print(f"Original duration: {final_state['duration']:.2f}s")
    print(f"Summary duration: {final_state['summary_clips_count']}s")
    
    try:
        display(Video(final_state['summary_video_path'], embed=True, width=640))
    except:
        print(f"Video saved but cannot display inline: {final_state['summary_video_path']}")

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)